In [1]:
MYPROJECT = '/home/john/work/myproject'
import os, sys
sys.path.insert(0, MYPROJECT)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "local_settings.py")
import django
django.setup()

In [2]:
import juridico.models as jmo

In [3]:
from gensim.models.doc2vec import *

In [4]:
import sqlite3
cappelCxn = sqlite3.connect("/home/mok/mining/jugements/cappel-mcgill/cappel.sqlite")
cc = cappelCxn.cursor()
educaloiCxn = sqlite3.connect("/home/mok/Sync/Projets/Juridico/educaloi.sqlite")
ce = educaloiCxn.cursor()

In [5]:
import treetaggerwrapper as ttw
tagger = ttw.TreeTagger(TAGLANG='fr')
bontags = "ADJ ADV NOM VER".split()

# Faire la liste de mots

In [6]:
%%time
from collections import Counter
cc.execute("select SansCitation from segments as s, jugements as j where j.NumEYB=s.NumEYB and j.lang='fr'")
cmots = Counter()
for t, in cc.fetchall():
    for k in ( tuple(j.strip() for j in i.split("\t")) for i in tagger.tag_text(t.strip()) ):
        if len(k) !=3: continue
        mot, tag, lex = k
        if tag[:3] in bontags:
            cmots.update([lex])

CPU times: user 5min 5s, sys: 2min 11s, total: 7min 16s
Wall time: 6min 10s


In [7]:
from nltk.corpus import stopwords
sw = stopwords.words("french")
mots = [ k for k,v in cmots.items() if v>4 and v<50000 and len(k)>1 and k not in sw]

In [8]:
len(mots)

14725

# Faire la liste de tags

In [9]:
tags_ls = []
for i, in cc.execute("select Indexation from jugements where lang='fr'").fetchall():
    tags_ls += [j.strip() for j in i.upper().split(";")]
tags_count = Counter(tags_ls)
tags_ls = [ k for k,v in tags_count.items() if v>10]
len(tags_ls)

705

In [10]:
from operator import itemgetter
import numpy as np
from scipy import sparse as sp

In [11]:
cc.execute("select rowid from jugements where lang='fr'")
domifs = [ i for i, in cc.fetchall() ]
len(domifs)

5229

In [12]:
data = []
row = []
col = []
for doc, ts in cc.execute("select rowid, Indexation from jugements where lang='fr'").fetchall():
    for t in ts.split(";"):
        tt = t.upper().strip()
        if tt not in tags_ls: continue
        data += [True]
        row += [tags_ls.index(tt)]
        col +=[domifs.index(doc)]

tags_mat = sp.csr_matrix((data,(row,col)), dtype=bool)

In [13]:
def is_parent(c, p):
    return np.all(np.logical_or(c, p) == p)

In [14]:
def find_parents(row):
    c = tags_mat[row]
    s = c.sum()
    cols = c.nonzero()[1]
    r = (tags_mat.T[cols].T.sum(axis=1)==s).nonzero()[0]
    return r[r!=row]

In [15]:
for t in tags_ls:
    c = Categorie(nom=t)
    c.save()

In [16]:
for i in range(len(tags_ls)):
    t = Categorie.objects.get(nom=tags_ls[i])
    for p in find_parents(i):
        t.parent.add(Categorie.objects.get(nom=tags_ls[p]))
    t.save()

# Apprendre le modèle doc2vec

In [115]:
import re
idx_sep = re.compile("\s*;\s*")

In [131]:
def get_corpus(cs):
    cs.execute("""
    select j.NumEYB, j.Indexation, group_concat(s.SansCitation, " " || char(10) || " ") from segments as s, jugements as j
where s.NumEYB = j.NumEYB
and j.lang='fr'
group by j.NumEYB
    """)
    for eyb, index, txt in cs.fetchall():
        t = [ tuple(j.strip() for j in i.split("\t")) for i in tagger.tag_text(txt.strip()) ]
        t = [ i for i in t if len(i) == 3]
        t = [ lex for mot, tag, lex in t if tag[:3] in bontags and lex in mots ]
        tags = [ "TAG_"+i for i in idx_sep.split(index.upper()) if i in tags_ls ] + [ "EYB_"+eyb.replace(" ","") ]
        
        yield TaggedDocument(t, tags)
        

In [132]:
def get_el_corpus(cs):
    cs.execute("select artid, texte from articles")
    for artid, txt in cs.fetchall():
        t = [ tuple(j.strip() for j in i.split("\t")) for i in tagger.tag_text(txt.strip()) ]
        t = [ i for i in t if len(i) == 3]
        t = [ lex for mot, tag, lex in t if tag[:3] in bontags and lex in mots ]
        tags = [ "EL_%d" % artid ]
        
        yield TaggedDocument(t, tags)

In [133]:
%%time
corpus = tuple(list(get_corpus(cc)) + list(get_el_corpus(ce)))

CPU times: user 8min 10s, sys: 1min 56s, total: 10min 6s
Wall time: 8min 59s


In [134]:
print(len(corpus))
print(corpus[0].words[:10])
print(corpus[0].tags)

5591
['moment', 'agent', 'diriger', 'véhicule', 'dernier', 'démarrer', 'faire', 'virage', 'phare', 'éteindre']
['TAG_PÉNAL', 'TAG_APPELS', "TAG_APPEL D'UNE CONDAMNATION OU D'UN ACQUITTEMENT", 'TAG_ACTE CRIMINEL', 'TAG_POUVOIRS DES POLICIERS', 'TAG_PREUVE PÉNALE', "TAG_SORTES D'ORDONNANCES", "TAG_REJET D'APPEL", 'EYB_EYB2004-81992']


In [135]:
%%time
d2v = Doc2Vec(documents=corpus, dm=0, size=100)

/home/mok/Python/py36/lib/python3.6/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


CPU times: user 3min 13s, sys: 0 ns, total: 3min 13s
Wall time: 1min 5s


In [136]:
d2v.docvecs.most_similar("EL_234", topn=500)

[('EL_252', 0.8398597240447998),
 ('EL_334', 0.8010889291763306),
 ('EL_233', 0.7912042737007141),
 ('EL_225', 0.7843679189682007),
 ('EL_331', 0.7766358256340027),
 ('EL_332', 0.7704771757125854),
 ('EL_187', 0.7621860504150391),
 ('EL_267', 0.7609560489654541),
 ('EL_302', 0.7562739849090576),
 ('EL_246', 0.7560646533966064),
 ('EL_192', 0.7501934170722961),
 ('EL_253', 0.7462419271469116),
 ('EL_226', 0.7368567585945129),
 ('EYB_EYB2006-111827', 0.736452043056488),
 ('EYB_EYB2008-150598', 0.7344065308570862),
 ('EL_258', 0.7329108119010925),
 ('EL_236', 0.7320713996887207),
 ('EL_255', 0.7277680039405823),
 ('EYB_EYB2006-109537', 0.7212036848068237),
 ('EL_185', 0.7187885046005249),
 ('EYB_EYB2007-126732', 0.7155494689941406),
 ('EL_242', 0.7146837115287781),
 ('EYB_EYB2007-120188', 0.7137342691421509),
 ('EL_204', 0.7130491733551025),
 ('EL_303', 0.7122701406478882),
 ('EL_260', 0.7093368768692017),
 ('EL_245', 0.7089210748672485),
 ('EYB_EYB2012-201565', 0.7082052230834961),
 ('EL

In [130]:
from scipy.spatial.distance import cosine, cdist

In [165]:
d2v.save("educaloi_cappel_d2v.model")

In [171]:
np.savez(
    "docvecs",
    docvectors = d2v.docvecs.vectors_docs,
    docnames = d2v.docvecs.index2entity
)


(6296, 100)